# Kunskapskontroll 2 Del 2, 
### PyEye (Realtids objektidetifiering)   av: Markus Tärning 2005

### Bakgrund till mitt projektarbete:
Min idé till detta projekt fick jag när jag läste om objektidetifiering i realtid. Jag tänkte då att det skulle kunna vara till stor hjälp för synskadade/blinda person att få höra vilka objekt de har framför sig. Min idé från början var att en person kan ha kameraglasögon eller action-kamera kopplad till en Raspberry Pie eller dylikt ihop med öronsnäckor. Men jag fick banta ned det hela lite och detta blev mitt resultat.




## Förklaring av vad de vitala delarna i koden gör:

Loop för att hålla programmet igång:
- Så länge running är True körs loopen kontinuerligt.

In [ ]:
while running:

Läs en bildruta från en videoström:

In [ ]:
ret, frame = cap.read()

Om en bildruta har lästs in korrekt:
- Endast om ret är True fortsätter programmet att bearbeta bildrutan.

In [ ]:
if ret:

Tidtagning för hur ofta prediktering ska göras (inferenstid):
- Startar en timer som används senare för att mäta hur lång tid processen tar.

In [ ]:
start_time = time.time()

Kör objektigenkänningsmodellen på bilden:
- model(frame) körs på den aktuella bildrutan.
- results innehåller de identifierade objekten i bilden.

In [ ]:
results = model(frame)

Skapar en tom mängd för att lagra upptäckta objekt:
- detected_objects används för att hålla reda på nya objekt som identifieras i denna iteration.

In [ ]:
detected_objects = set()

Iterera över modellens resultat:
- results kan innehålla flera objektupptäckter, itererar igenom dessa.

- Iterera över de identifierade objekten i varje resultat:

- Varje box innehåller information om varje upptäckt objekt.

In [ ]:
for box in result.boxes:

Extrahera klassificeringssäkerheten (confidence score):
- box.conf[0] innehåller förtroendegraden (sannolikheten) för att detta är ett korrekt identifierat objekt.
- .item() konverterar från en tensor till en vanlig Python-float.

In [ ]:
conf = box.conf[0].item()

Om säkerheten överstiger en tröskel:
- Jämför förtroendegraden med en förbestämd confidence_threshold.
- Endast objekt med en säkerhet över tröskeln behandlas vidare.

In [ ]:
if conf >= confidence_threshold:

Extrahera koordinater och klassetikett
- box.xyxy[0] innehåller koordinater för objektets bounding box (vänster-övre och höger-nedre hörn).
- result.names[int(box.cls[0])] hämtar det textnamn som motsvarar klassetiketten.

In [ ]:
x1, y1, x2, y2 = map(int, box.xyxy[0])
label = result.names[int(box.cls[0])]

Kolla om objektet ska räknas som nyupptäckt:
- Om no_repeat är False läggs alltid objektet till.
- Om no_repeat är True läggs objektet endast till om det inte redan finns i identified_objects.

In [ ]:
if not no_repeat or label not in identified_objects:
    detected_objects.add(label)

Rita en rektangel runt detekterade objektet:
- En grön (0, 255, 0) rektangel ritas runt detekterade objektet.

In [ ]:
cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

Lägger till text med objektets namn och klassifiseringsvärde:
- Texten innehåller objektets namn och klassifiseringsvärde (t.ex. "Katt 0.87" = modellen tycker att det till 87% chans är en katt).
- Skriften är i font FONT_HERSHEY_SIMPLEX, storlek 0.5, färg grön (0, 255, 0) och tjocklek 2.

In [ ]:
cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10), 
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

Lägg till identifierade objekt i den globala listan:
- Uppdaterar identified_objects med de nyupptäckta objekten.

In [ ]:
identified_objects.update(detected_objects)

Om talsyntes är aktiverad och nya objekt har upptäckts:
- Kollar om speak_enabled är True och om några nya objekt har identifierats.

In [ ]:
if speak_enabled and detected_objects:

Om de nyupptäckta objekten skiljer sig från senaste gången:
- spoken_text = ", ".join(detected_objects)
if spoken_text != last_spoken:
- Om objektlistan har ändrats sedan förra iterationen.


In [ ]:
spoken_text = ", ".join(detected_objects)
if spoken_text != last_spoken:

Läs upp de identifierade objekten med talsyntes:
- engine.say(spoken_text) skickar texten till talsyntesen.
- engine.runAndWait() gör att programmet väntar tills talet är färdigt innan det fortsätter.
- last_spoken uppdateras för att undvika upprepningar.

In [ ]:
engine.say(spoken_text)
engine.runAndWait()
last_spoken = spoken_text

Konvertera bilden från BGR till RGB för korrekt färgåtergivning:
- OpenCV använder BGR-färger, men PIL/Tkinter använder sig av RGB.

In [ ]:
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

Konvertera bilden till PIL-format:
- Image.fromarray(frame) konverterar den OpenCV-bearbetade bilden till en PIL-bild.

In [ ]:
img = Image.fromarray(frame)

Skapa en Tkinter-kompatibel bild:
- ImageTk.PhotoImage(image=img) skapar en bild som kan visas i ett Tkinter-grafiskt gränssnitt.

In [ ]:
imgtk = ImageTk.PhotoImage(image=img)

Uppdatera Tkinter-canvas med den nya bilden:
- canvas.imgtk = imgtk ser till att bilden inte rensas bort av Python:s garbage collector.
- canvas.configure(image=imgtk) uppdaterar gränssnittet med den nya bilden.

In [ ]:
canvas.imgtk = imgtk
canvas.configure(image=imgtk)

### Sammanfattning:
- Koden fångar en videoström från datorns kamera, skickar sedan varje bildruta till en objektigenkänningsmodell, markerar identifierade objekt visuellt, läser upp dem med talsyntes om inställt, och visar den bearbetade bilden i ett Tkinter-fönster.
- Tidsintervall för hur ofta modellen ska prediktera och sannolikhetströskel är justerbara.
- Listan med identifierade objekt kan skrivas ut.

### Koden i sin helhet ligger på min githubsida: https://github.com/MarkusTarning/code_lib//PyEye.py
- Jag har gjort en demonstration i Mp4-format som jag kan visa eller skicka om så önskas.

### När du lämnar in kunskapskontrollen ska du även besvara följande reflektionsfrågor:  
- 1. Hur har du upplevt arbetet med projektet?  
- 2. Har något varit utmanande, hur har du hanterat det? Vilka lärdomar har du tagit?  
- 3. Vilket betyg anser du att du ska ha?  
På denna kunskapskontroll kan man få betygen IG/G/VG.

1.) Arbetet har varit mycket lärorikt, givande och roligt. Jag kommer förmodligen bygga vidare på detta projekt samt fortsätta arbetet med cancerbilderna.

2.) Jag började mitt projekt med ett väldigt stort dataset bestående av Dicom-bilder (bröstcancer). Jag kände rätt så snart att det blev en alldeles för stor uppgift att hinna med inom utsatt tid. Så mina lärdomar av detta är att försöka lägga sig på en nivå som är hanterbar. 
##### Andra utmaningar:
- Då modellen är väldigt resurskrävande så jag har kämpat mycket med att datorn har hägt sig. Samt kompatibilitetsproblem mellan olika Python-moduler och versioner.
- I och med att programmet hela tiden itererar i sin videoström så har det varit en hel del jobb med att lägga funktioner och variabler i rätt loop.
- Har aldrig tidigare varit i kontakt med på med CV2, TKinter eller Yolo med dess videoströmning, ganska utmanande!
- Hade en del strul med: BGR och RGB innan jag kom på det.
##### Lärdomar:
- Det mesta i detta projekt var för mig nytt, så det har varit lärorikt.
- Att göra proof of concept innan man satsar allt för mycket tid. Många grejer visar sig rätt snart om de är möjliga. Önskar att jag hade gjort det med cancerbilderna, vilket hade besparat mig mycket dyrbar tid.

3.) Jag anser att betyget VG bör vara uppfyllt med detta projektarbete.